In [1]:
import pygame
import sys
import numpy as np
import random
import pandas as pd
import time
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
BATCH_SIZE=128


pygame 2.1.2 (SDL 2.0.18, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
width=10
height=10
pixel_size=(50,50)
display_size=(pixel_size[0]*width,pixel_size[1]*height)
record=[]
game_num=1
p=0
step=0

color_dict={
    "wall":(177,177,177),
    "target":(0,177,0),
    "normal":(255,255,255),
    "start":(177,0,0)
}

map_dict={
    "target":(7,2),
    "start":(2,height-1),
    "tank1":(7,4),
    "tank2":(5,2),
    "tank3":(random.randint(3,width-3),random.randint(2,height-3)),

   }

wall_location=[]
for w in range(width):
    wall_location.append((w,0))
    wall_location.append((w,height-1))
    
for h in range(height):    
    wall_location.append((0,h))
    wall_location.append((width-1,h))
    
wall_location.remove(map_dict.get("start"))
wall_location.append((map_dict.get("start")[0],map_dict.get("start")[1]+1))
map_dict['wall']=wall_location

df100=pd.DataFrame(columns=['s','a','s_','r'])
df1=pd.DataFrame(columns=['s','a','s_','r'])
df0=pd.DataFrame(columns=['s','a','s_','r'])

epsilon=0
reward=[-0.02]
gamma=0.9
a_dict={
    0:'up',
    1:'down',
    2:'left',
    3:'right'
}

s_index={}
for i in range(width-2):
    for j in range(height-2):
        s_index.update({(i+1,j+1):(i+(j*(width-2)))})

s_index.update({map_dict.get("start"):(width-2)*(height-2)})   


In [3]:
class tank(pygame.sprite.Sprite):
    def __init__(self,filename,location):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load(filename).convert()
        self.image = pygame.transform.scale(self.image,(50,50))

        self.rect = self.image.get_rect()
        self.rect.topleft=(location[0]*pixel_size[0],location[1]*pixel_size[1])
        
        
def draw_map():
    for i in range(width*height):
        w=i-width*(i//width)
        h=i//width
        
        if (w,h)==map_dict.get("target"):
            color=color_dict.get("target")
        elif (w,h)==map_dict.get("start"):
            color=color_dict.get("start")
        elif (w,h) in map_dict.get("wall"):
            color=color_dict.get("wall")
        else:
            color=color_dict.get("normal")
        
        pygame.draw.rect(game_map,color,(w*50,h*50,pixel_size[0],pixel_size[1]))

        
def crash_wall():
    if (person_site[0],person_site[1]) in map_dict.get("wall"):
        return True
    else:
        return False
    
    
def crash_detect(sprite1,sprite2):
    global game_num
    global game_state
    result=pygame.sprite.collide_mask(sprite1,sprite2)
    
    if result:
        if sprite2 in tank1_group:
            print("这是第{}局游戏，我死了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(0)
            reward.append(-1)

        elif sprite2 in princess_group:
            print("这是第{}局游戏，我赢了".format(game_num))
            if game_num>20:
                print('最后20盘的胜率为{}'.format(record[-20:].count(1)/20))
            record.append(1)
            reward.append(1)

        game_num+=1
        game_state=1
        
        
def move(direction):
    global person_site
    global step
    if direction=='up':
        person_site[1]-=1
        a= crash_wall()
        if a==True:
            person_site[1]+=1
        
    if direction=='down':
        person_site[1]+=1
        a= crash_wall()
        if a==True:
            person_site[1]-=1
    if direction=='left':
        person_site[0]-=1
        a= crash_wall()
        if a==True:
            person_site[0]+=1
    if direction=='right':
        person_site[0]+=1
        a= crash_wall()
        if a==True:
            person_site[0]-=1
        
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    crash_detect(person,tank1)       
    crash_detect(person,tank2)   
    crash_detect(person,tank3)       

    crash_detect(person,princess)         
    step+=1

In [4]:
def pick_a(state):    
    if random.random()>epsilon:
        action=random.randint(0,3)
    else:
        x=model1.predict([state])
        action= x.argmax()
    return action


def one_turn():
    s1=int(person_site[0])
    s2=int(person_site[1])
    a=pick_a([s1,s2])
    move(a_dict.get(a))
    s1_=int(person_site[0])
    s2_=int(person_site[1])
    r=reward[-1]
    trace_collect(s1,s2,a,s1_,s2_,r)
    
    if game_num>50 and step%200==0:
        fit(10)
        
    if game_state==1:           
        reset()

        
def trace_collect(s1,s2,a,s1_,s2_,r):   
    global df100
    global df1
    global df0
    new_trace={
        's1':s1,
        's2':s2,
        'a':a,
        's1_':s1_,
        's2_':s2_,
        'r':r
    }
    if r==1:
        df100=df100.append(new_trace,ignore_index=True)
        df100=df100.iloc[-100000:]
        #df100.to_csv('trace100.csv')
    if r==-1:
        df1=df1.append(new_trace,ignore_index=True)
        df1=df1.iloc[-100000:]
        #df1.to_csv('trace1.csv')
    else:
        df0=df0.append(new_trace,ignore_index=True)
        df0=df0.iloc[-100000:]
        #df0.to_csv('trace0.csv')
    
    
def preprocessing(batch_size):
    #df100=pd.read_csv('trace100.csv')
    #df1=pd.read_csv('trace1.csv')
    #df0=pd.read_csv('trace0.csv')
    ds100=df100.sample(batch_size,replace = True)
    ds1=df1.sample(batch_size//2,replace = True)
    ds0=df0.sample(batch_size//2,replace = True)
    train_ds=pd.concat((ds100,ds1,ds0))
    train_ds=train_ds.sample(batch_size)
    return train_ds        

In [5]:
def train_step(train_ds):
    global p
    s = np.int32(np.stack((train_ds['s1'],train_ds['s2']),1))
    a = np.array(train_ds['a'])
    s_ = np.int32(np.stack((train_ds['s1_'],train_ds['s2_']),1))
    r = np.array(train_ds['r'])
    q_=model1(s_)
    a_=np.argmax(q_,axis=1)

    
    c1=np.zeros((len(train_ds),4))   #A方向外全为0的矩阵
    for i in range(len(a)):
        c1[i][int(a[i])]=1
    
    c2=np.ones((len(train_ds),4))    #A方向外全为1的矩阵
    for i in range(len(a)):
        c2[i][int(a[i])]=0
        
    c3=np.zeros((len(train_ds),4))   #A_方向外全为0的矩阵
    for i in range(len(a_)):
        c3[i][int(a_[i])]=1
        
    c4=np.ones((len(train_ds),4))   #A_方向外全为1的矩阵
    for i in range(len(a_)):
        c4[i][int(a_[i])]=0
    
    d=np.ones(len(r))
    for i in range(len(r)):
        if r[i]==1:
            d[i]=0
        elif r[i]==-1:
            d[i]=0
        else:
            pass
        
    qloss=model1(s)
    qt=model2(s_)   
    #qtarget=tf.reduce_max(qt,axis=1)*gamma*d+r
    qt=tf.reduce_max(qt*c3,axis=1)
    qt=qt*gamma*d+r
    qtloss=tf.transpose([qt,qt,qt,qt])*c1+qloss*c2
       
    with tf.GradientTape(persistent=True) as tape:
        #loss函数
        q=model1(s)        
        loss=tf.reduce_sum(tf.square(qtloss-q))
        p=float(loss)
    
    gradient=tape.gradient(loss,model1.trainable_variables)
    opt.apply_gradients(zip(gradient,model1.trainable_variables))
    del tape    

In [6]:
def DDDQN():
    input=tf.keras.layers.Input(shape=((2)))
    x=tf.keras.layers.Dense(128,activation='relu')(input)
    x=tf.keras.layers.Dense(128,activation='relu')(x)
    x=tf.keras.layers.Dense(128,activation='relu')(x)
    x=tf.keras.layers.Dense(128,activation='relu')(x)
    x=tf.keras.layers.Dense(128,activation='relu')(x)
    
    v=tf.keras.layers.Dense(128,activation='relu')(x)
    v=tf.keras.layers.Dense(128,activation='relu')(v)
    v=tf.keras.layers.Dense(1)(v)
    
    a=tf.keras.layers.Dense(128,activation='relu')(x)
    a=tf.keras.layers.Dense(128,activation='relu')(a)
    a=tf.keras.layers.Dense(4)(a)
    
    output=v+a-tf.reduce_max(a)
    
    model=tf.keras.Model(inputs=input,outputs=output)
    return model    
    

In [7]:
model1 =DDDQN()
model2 =DDDQN()

opt=tf.keras.optimizers.Adam(learning_rate=0.001)

In [8]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          384         input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 128)          16512       dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          16512       dense_1[0][0]                    
______________________________________________________________________________________________

In [9]:
def fit(epochs):
    for epoch in range(epochs):
        train_ds=preprocessing(BATCH_SIZE)
        for i in range(50):
            train_step(train_ds)
        print(str(p)+'.',end='')
        
        if (epoch+1)%10==0:
            print("EPOCH:",epoch+1)

    
def load_weight():
    model1.save_weights('DQN.h5')
    model2.load_weights('DQN.h5')
    
    
def reset():
    global person_site
    global game_state
    person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
    person.rect=(person_site[0]*pixel_size[0],person_site[1]*pixel_size[1],50,50)
    game_state=0
    reward.append(-0.02)
    
    if game_num>70 and game_num%20==0:    
        load_weight()
        #debug()
        

def debug():
    for i in range(width-2):
        for j in range(height-2):
            result=model1.predict([[i+1,j+1]])
            print(i+1,j+1)
            print(result)

In [10]:
pygame.init()
clock = pygame.time.Clock()
pygame.display.set_caption("AI训练游戏")

screen = pygame.display.set_mode(display_size)
game_map=pygame.surface.Surface(display_size)
draw_map()

tank1_group = pygame.sprite.Group()
tank2_group = pygame.sprite.Group()
princess_group = pygame.sprite.Group()
person_group=pygame.sprite.Group()

tank1 = tank("img/tank.jpg",map_dict.get("tank1"))
tank2 = tank("img/tank.jpg",map_dict.get("tank2"))
tank3 = tank("img/tank.jpg",map_dict.get("tank3"))

tank1_group.add(tank1)
tank1_group.add(tank2)
tank1_group.add(tank3)


princess=tank('img/sidalin.jpeg',map_dict.get("target"))
princess_group.add(princess)

person_site=[map_dict.get("start")[0],map_dict.get("start")[1]]
person=tank('img/maintank.jpeg',person_site)
person_group.add(person)

reset()

while True:
    clock.tick(60)

    screen.blit(game_map,(0,0))
    tank1_group.draw(screen)
    princess_group.draw(screen)
    screen.blit(person.image,person.rect)
    tank2_group.draw(screen)
    
    
    one_turn()



    if game_num==50:
        epsilon=0.2
    elif game_num==100:
        epsilon=0.3   
    elif game_num==150:
        epsilon=0.4
    elif game_num==200:
        epsilon=0.5
    elif game_num==250:
        epsilon=0.6
    elif game_num==275:
        epsilon=0.7
    elif game_num==300:
        epsilon=0.8
    elif game_num==325:
        epsilon=0.9
    elif game_num==350:
        epsilon=0.95
    elif game_num==375:
        epsilon=0.99
        
    for event in pygame.event.get():
        if event.type ==pygame.QUIT:
            pygame.quit()
            sys.exit()
            

        
#       if event.type ==pygame.KEYDOWN:
#           if event.key == pygame.K_UP:
#               move('up')
#           if event.key == pygame.K_DOWN:
#               move('down')
#           if event.key == pygame.K_LEFT:
#               move('left')
#           if event.key == pygame.K_RIGHT:
#               move('right')
#           if event.key == pygame.K_SPACE:
                
                
     
    pygame.display.update()

这是第1局游戏，我死了
这是第2局游戏，我死了
这是第3局游戏，我死了
这是第4局游戏，我赢了
这是第5局游戏，我死了
这是第6局游戏，我死了
这是第7局游戏，我死了
这是第8局游戏，我死了
这是第9局游戏，我死了
这是第10局游戏，我死了
这是第11局游戏，我死了
这是第12局游戏，我死了
这是第13局游戏，我死了
这是第14局游戏，我死了
这是第15局游戏，我死了
这是第16局游戏，我死了
这是第17局游戏，我死了
这是第18局游戏，我死了
这是第19局游戏，我死了
这是第20局游戏，我死了
这是第21局游戏，我赢了
最后20盘的胜率为0.05
这是第22局游戏，我死了
最后20盘的胜率为0.1
这是第23局游戏，我死了
最后20盘的胜率为0.1
这是第24局游戏，我死了
最后20盘的胜率为0.1
这是第25局游戏，我死了
最后20盘的胜率为0.05
这是第26局游戏，我死了
最后20盘的胜率为0.05
这是第27局游戏，我死了
最后20盘的胜率为0.05
这是第28局游戏，我死了
最后20盘的胜率为0.05
这是第29局游戏，我死了
最后20盘的胜率为0.05
这是第30局游戏，我死了
最后20盘的胜率为0.05
这是第31局游戏，我死了
最后20盘的胜率为0.05
这是第32局游戏，我死了
最后20盘的胜率为0.05
这是第33局游戏，我死了
最后20盘的胜率为0.05
这是第34局游戏，我死了
最后20盘的胜率为0.05
这是第35局游戏，我死了
最后20盘的胜率为0.05
这是第36局游戏，我死了
最后20盘的胜率为0.05
这是第37局游戏，我赢了
最后20盘的胜率为0.05
这是第38局游戏，我死了
最后20盘的胜率为0.1
这是第39局游戏，我死了
最后20盘的胜率为0.1
这是第40局游戏，我死了
最后20盘的胜率为0.1
这是第41局游戏，我死了
最后20盘的胜率为0.1
这是第42局游戏，我死了
最后20盘的胜率为0.05
这是第43局游戏，我赢了
最后20盘的胜率为0.05
这是第44局游戏，我死了
最后20盘的胜率为0.1
这是第45局游戏，我死了
最后20盘的胜率为0.1
这是第46局游戏，我死了
最后20盘的胜率为0.1
这是第47局游戏，我死了
最后20盘的胜率为0.1
这是第48局游戏，我死了
最后20盘的胜率为0.1
这是第49

0.0040900250896811485.0.009891681373119354.0.0017739309696480632.0.004267911892384291.6.517935253214091e-05.0.0026863478124141693.0.004839201923459768.0.0013373844558373094.0.0015530474483966827.0.009267725050449371.EPOCH: 10
0.0017073588678613305.0.0005387018318288028.0.0022656789515167475.0.0004948715795762837.0.0004674728843383491.0.001194310374557972.0.0017780164489522576.0.0016234314534813166.0.00023696503194514662.0.050681017339229584.EPOCH: 10
0.00036932731745764613.0.007036250084638596.0.00021189515246078372.0.0010153820039704442.0.0009068163926713169.0.0005748904077336192.0.0002622075262479484.0.0007955652545206249.0.00018171123520005494.0.00015702388191130012.EPOCH: 10
这是第102局游戏，我死了
最后20盘的胜率为0.05
0.00029677717247977853.0.0006479559815488756.0.0025481816846877337.0.0009895903058350086.0.0016532703302800655.0.0007031620480120182.0.017942676320672035.0.032238349318504333.0.0006469232612289488.0.00163992028683424.EPOCH: 10
这是第103局游戏，我死了
最后20盘的胜率为0.05
0.04429072514176369.0.0020972

0.0004902350483462214.0.0003757814993150532.0.027607601135969162.0.02891627512872219.0.03396492451429367.0.01328889187425375.0.009280783124268055.0.0032847090624272823.0.010699559934437275.0.02780270390212536.EPOCH: 10
这是第134局游戏，我死了
最后20盘的胜率为0.3
0.002176179550588131.0.00720274168998003.0.002103831386193633.0.41670477390289307.0.03847413510084152.0.021192852407693863.0.0403837226331234.0.020473401993513107.0.004762918688356876.0.013208393938839436.EPOCH: 10
这是第135局游戏，我死了
最后20盘的胜率为0.3
0.0034536339808255434.0.0006381107959896326.0.022513577714562416.0.008563293144106865.0.006551117170602083.0.016086243093013763.0.051348015666007996.0.023688368499279022.0.0021480924915522337.0.00018956983694806695.EPOCH: 10
0.028547249734401703.0.02252594381570816.0.010978997685015202.0.02670864202082157.0.0031164935790002346.0.0007527519483119249.0.03357783332467079.0.0011611598310992122.0.00528380274772644.0.0015447806799784303.EPOCH: 10
0.0452665351331234.0.0014275937573984265.0.004010596312582493.0.001

0.013632658869028091.0.0040230099111795425.0.0028212955221533775.0.14774367213249207.0.032060641795396805.0.05052804946899414.0.005371458362787962.0.03751150518655777.0.02299077808856964.0.07487057894468307.EPOCH: 10
这是第191局游戏，我死了
最后20盘的胜率为0.3
这是第192局游戏，我死了
最后20盘的胜率为0.3
这是第193局游戏，我死了
最后20盘的胜率为0.3
这是第194局游戏，我死了
最后20盘的胜率为0.3
这是第195局游戏，我死了
最后20盘的胜率为0.3
0.07795591652393341.0.14924690127372742.0.0970936194062233.0.0186886265873909.0.18843024969100952.0.034337569028139114.0.11627254635095596.0.034427348524332047.0.0015305676497519016.0.19242429733276367.EPOCH: 10
这是第196局游戏，我赢了
最后20盘的胜率为0.3
这是第197局游戏，我死了
最后20盘的胜率为0.3
这是第198局游戏，我死了
最后20盘的胜率为0.25
这是第199局游戏，我死了
最后20盘的胜率为0.25
这是第200局游戏，我死了
最后20盘的胜率为0.25
这是第201局游戏，我赢了
最后20盘的胜率为0.25
0.16421934962272644.0.20341713726520538.0.002227450953796506.0.005904969293624163.0.19135534763336182.0.24426527321338654.0.1322091817855835.0.13733728229999542.0.26237553358078003.0.2840383052825928.EPOCH: 10
这是第202局游戏，我死了
最后20盘的胜率为0.25
这是第203局游戏，我赢了
最后20盘的胜率为0.2
这是第20

0.06758818030357361.0.004220106638967991.0.3667479455471039.0.036821767687797546.0.11564023047685623.0.0035723086912184954.0.31285837292671204.0.04075135290622711.0.022581469267606735.0.601747453212738.EPOCH: 10
这是第287局游戏，我死了
最后20盘的胜率为0.5
0.05234675854444504.0.05656114220619202.0.01584433764219284.0.09636325389146805.0.30722367763519287.0.07614199072122574.0.2740975618362427.0.025591164827346802.0.11792153120040894.0.01305987499654293.EPOCH: 10
这是第288局游戏，我赢了
最后20盘的胜率为0.5
这是第289局游戏，我死了
最后20盘的胜率为0.55
这是第290局游戏，我赢了
最后20盘的胜率为0.55
0.02330799400806427.0.009657012298703194.0.16442111134529114.0.14742526412010193.0.0648152083158493.0.03872902691364288.0.09069617092609406.0.13267706334590912.0.09321320056915283.0.009667902253568172.EPOCH: 10
这是第291局游戏，我赢了
最后20盘的胜率为0.55
这是第292局游戏，我赢了
最后20盘的胜率为0.6
这是第293局游戏，我赢了
最后20盘的胜率为0.6
这是第294局游戏，我死了
最后20盘的胜率为0.6
这是第295局游戏，我赢了
最后20盘的胜率为0.6
0.029274092987179756.0.034217119216918945.0.011288636364042759.0.0032771287951618433.0.07114658504724503.0.09690947085618

...

0.022198524326086044.0.0026544989086687565.0.041407134383916855.0.07351049780845642.0.04441205784678459.0.0258912593126297.0.1359989047050476.0.0026485202834010124.0.005941151175647974.0.011932039633393288.EPOCH: 10
0.006173077505081892.0.06926009058952332.0.05005887895822525.0.10319308191537857.0.01707299053668976.0.017122367396950722.0.010861259885132313.0.037123147398233414.0.08826948702335358.0.026652464643120766.EPOCH: 10
0.09312763065099716.0.029725588858127594.0.08999434113502502.0.007355613633990288.0.03795471414923668.0.007685174234211445.0.033263199031353.0.1429750770330429.0.01929258182644844.0.007592879701405764.EPOCH: 10
这是第446局游戏，我赢了
最后20盘的胜率为0.95
0.024454161524772644.0.029239825904369354.0.00252373144030571.0.029318174347281456.0.007366416975855827.0.012027367018163204.0.01746845617890358.0.07464234530925751.0.09098140895366669.0.05387180671095848.EPOCH: 10
0.007590907625854015.0.03972064331173897.0.1107218936085701.0.010749900713562965.0.010441020131111145.0.02540189959

0.0018593647982925177.0.01712879166007042.0.010892211459577084.0.05528385564684868.0.02189478650689125.0.04962417110800743.0.011525717563927174.0.06342575699090958.0.000994581263512373.0.008780284784734249.EPOCH: 10
0.0059039173647761345.0.007105040363967419.0.006495167501270771.0.003852784400805831.0.037433329969644547.0.0236503966152668.0.04145233333110809.0.012914741411805153.0.08566305041313171.0.01039254479110241.EPOCH: 10
这是第456局游戏，我赢了
最后20盘的胜率为0.95
0.027343595400452614.0.033294353634119034.0.08319874852895737.0.002933210227638483.0.012062285095453262.0.026916896924376488.0.012843172065913677.0.038903795182704926.0.0026521014515310526.0.1882944405078888.EPOCH: 10
0.014664913527667522.0.014926698058843613.0.0025693222414702177.0.004040516912937164.0.2191888391971588.0.01658259704709053.0.004517178516834974.0.051443081349134445.0.00772654777392745.0.0034640314988791943.EPOCH: 10
0.03356073796749115.0.005980960093438625.0.01376674696803093.0.03702597692608833.0.012763078324496746.0.

0.004848487209528685.0.0055887093767523766.0.002095272298902273.0.0006490912055596709.0.00038694555405527353.0.008102085441350937.0.0011611159425228834.0.002368294633924961.0.0065437499433755875.0.037019435316324234.EPOCH: 10
0.030291007831692696.0.5248048305511475.0.16939876973628998.0.02691204845905304.0.08836333453655243.0.4125041663646698.0.09615812450647354.0.02174779772758484.0.03963880240917206.0.020856883376836777.EPOCH: 10
0.24863719940185547.0.033461444079875946.0.0089237280189991.0.0007801718893460929.0.017341120168566704.0.07969654351472855.0.005345665384083986.0.0016468472313135862.0.0013623916311189532.0.004439687356352806.EPOCH: 10
0.07167033106088638.0.1762004941701889.0.00786774605512619.0.0018727966817095876.0.003650153521448374.0.002725175116211176.0.02140577882528305.0.003506404347717762.0.00036776234628632665.0.002704222220927477.EPOCH: 10
0.0019486225210130215.0.035708121955394745.0.005626905709505081.0.004721555858850479.0.006933117751032114.0.03367603197693825.0

SystemExit: 

d:\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
